In [7]:
import uuid
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForQuestionAnswering, pipeline
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableConfig
from langchain_core.chat_history import BaseChatMessageHistory


df = pd.read_csv('C:\\Users\\Leo\\code\\ch07_트랜스포머_RAG\\test03\\data\\data.csv', encoding='utf8')
texts = df["text"].tolist()
docs = [Document(page_content=text) for text in texts]

embdedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = Chroma.from_documents(
    documents=docs, embedding=embdedding_model, 
    persist_directory="./chroma_db3")

retriever = Chroma(
    persist_directory="./chroma_db3",
    embedding_function=embdedding_model
).as_retriever(search_kwargs={"k":3})

model_id = "monologg/koelectra-base-v3-finetuned-korquad"
qa_tokenizer = AutoTokenizer.from_pretrained(model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_id)
qa_pipeline = pipeline(
    "question-answering",
    model = qa_model,
    tokenizer = qa_tokenizer,
    device =-1   
)
qa_llm = HuggingFacePipeline(pipeline=qa_pipeline)

rewrite_model_id = "skt/kogpt2-base-v2"
rewrite_tokenizer = AutoTokenizer.from_pretrained(rewrite_model_id)
rewrite_model = AutoModelForCausalLM.from_pretrained(rewrite_model_id)
rewrite_tokenizer.model_max_length = 1024

rewrite_pipeline = pipeline(
    "text-generation",
    model = rewrite_model,
    tokenizer = rewrite_tokenizer,
    max_new_tokens = 64,
)

rewrite_llm = HuggingFacePipeline(pipeline=rewrite_pipeline)
chats_by_ssetion_id = {}

def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chats_by_ssetion_id:
        chats_by_ssetion_id[session_id] = InMemoryChatMessageHistory()
    return chats_by_ssetion_id[session_id]


def ask_question(state, config:RunnableConfig):
    session_id = config["configurable"]["session_id"]
    history = get_chat_history(session_id)

    question = input("질문을 입력해주세요:").strip()
    history.add_user_message(question)
    
    return {
        "question": question,        
        "original_question": question,
        "attempt":1,
        "history": history
    }

def search_and_answer(state):
    question = state["question"]
    docs = retriever.invoke(question)
    context = "\n".join([doc.page_content for doc in docs])
    result = qa_pipeline(
        question=question, context=context)
    return {**state, "docs": docs, "context": context,"answer": result["answer"]}

def assess_answer_quality(state):
    answer = state["answer"].strip()
    attempt = state.get("attempt", 1)
    if len(answer) < 10 and attempt <3:
        print("not enough answer, try again.")
        return {**state, "needs_followup": True}
    print(f"\n질문: {state['question']}\n답변:{answer}")
    return {**state, "needs_followup": False}

def generate_followup(state):
    original_question = state["original_question"]
    prompt = f"original question: {original_question}\n\더 구체적으로 묻기 위해 재질문을 만들어주세요"
    followup = rewrite_pipeline(prompt)[0]["generated_text"]
    print(f"\n재질문: {followup.strip()}")
    return {
        **state,
        "question": followup.strip(),
        "attempt": state["attempt"] + 1
    }

def finalize(state):
    session_id = state.get("session_id")
    state["history"].add_ai_message(state["answer"])
    return state

def ask_continue(state):
    cont = input("\n계속하시겠습니까? (y/n): ").strip()
    state["continue"] = cont.startswith("y")
    return state

def should_followup(state):
    return "generate_followup" if state.get("needs_followup") else "finalize"

def should_continue(state):
    return "ask_question" if state.get("continue")  else END


graph = StateGraph(dict)
graph.add_node("ask_question", ask_question)
graph.add_node("search_and_answer", search_and_answer)
graph.add_node("assess_answer_quality", assess_answer_quality)
graph.add_node("generate_followup", generate_followup)
graph.add_node("finalize", finalize)
graph.add_node("ask_continue", ask_continue)

graph.set_entry_point("ask_question")

graph.add_edge("ask_question", "search_and_answer")
graph.add_edge("search_and_answer", "assess_answer_quality")
graph.add_conditional_edges("assess_answer_quality", should_followup, {
    "generate_followup": "generate_followup",
    "finalize": "finalize"
})

graph.add_edge("generate_followup", "search_and_answer")
graph.add_edge("finalize", "ask_continue")
graph.add_conditional_edges("ask_continue", should_continue, {
    "ask_question": "ask_question",
    END: END
})

session_id = str(uuid.uuid4())
config = {"configurable": {"session_id": session_id}}
app = graph.compile()
app.invoke({}, config=config)


Device set to use cpu
Device set to use cpu


not enough answer, try again.

재질문: original question: 공정 조건 이상 발생 시 어떻게 처리하나요?
\더 구체적으로 묻기 위해 재질문을 만들어주세요!
\더 구체적으로 묻기 위해 재질문 만들기!
\더 구체적으로 질문하기 위해 재질문 만들기!
\더 구체적으로 질문하기 위해 재질문 만들기!
\더 구체적으로 질문하기 위해 재질문 만들기!
\더 구체적으로 질문하기 위해 재질문 만들기!
\더 구체적으로 질문하기 위해서 재질
not enough answer, try again.

재질문: original question: 공정 조건 이상 발생 시 어떻게 처리하나요?
\더 구체적으로 묻기 위해 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문 대신 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문을 만들어주세요!
\더 구체적으로

질문: original question: 공정 조건 이상 발생 시 어떻게 처리하나요?
\더 구체적으로 묻기 위해 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문 대신 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문을 만들어주세요!
\더 구체적으로 대답하기 위해 재질문을 만들어주세요!
\더 구체적으로
답변:검사 장비는 매일 초기화 후 기능 점검을


{'question': 'original question: 공정 조건 이상 발생 시 어떻게 처리하나요?\n\\더 구체적으로 묻기 위해 재질문을 만들어주세요!\n\\더 구체적으로 대답하기 위해 재질문 대신 재질문을 만들어주세요!\n\\더 구체적으로 대답하기 위해 재질문을 만들어주세요!\n\\더 구체적으로 대답하기 위해 재질문을 만들어주세요!\n\\더 구체적으로 대답하기 위해 재질문을 만들어주세요!\n\\더 구체적으로',
 'original_question': '공정 조건 이상 발생 시 어떻게 처리하나요?',
 'attempt': 3,
 'history': InMemoryChatMessageHistory(messages=[HumanMessage(content='공정 조건 이상 발생 시 어떻게 처리하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='검사 장비는 매일 초기화 후 기능 점검을', additional_kwargs={}, response_metadata={})]),
 'docs': [Document(id='8bde1239-dfad-4a24-a67e-d48a3c423905', metadata={}, page_content='검사 장비는 매일 초기화 후 기능 점검을 실시합니다.'),
  Document(id='24d1021a-8038-4944-81c7-15744521f2ec', metadata={}, page_content='검사 장비는 매일 초기화 후 기능 점검을 실시합니다.'),
  Document(id='93fa7d22-941d-426b-a69d-7779fce64e7b', metadata={}, page_content='검사 장비는 매일 초기화 후 기능 점검을 실시합니다.')],
 'context': '검사 장비는 매일 초기화 후 기능 점검을 실시합니다.\n검사 장비는 매일 초기화 후 기능 점검을 실시합니다.\n검사 장비는 매일 초기화 후 기능 점검을 실시합니다.',
 'answe